<center>
<h4>CDS 110, Lecture 9</h4>
<font color=blue><h1>PID Control of a Servomechanism</h1></font>
<h3>Richard M. Murray, Winter 2024</h3>
</center>

[Open in Google Colab](https://colab.research.google.com/drive/1BP0DFHh94tSxAyQetvOEbBEHKrSoVGQW)

In this lecture we will use a variety of methods to design proportional (P), proportional-integral (PI), and proportional-integral-derivative (PID) controllers for a cart pendulum system.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi
try:
  import control as ct
  print("python-control", ct.__version__)
except ImportError:
  !pip install control
  import control as ct

## System dynamics

Consider a simple mechanism consisting of a spring loaded arm that is driven by a  motor, as shown below:

<center><img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/servomech-diagram.png" width=200 alt="servomech-diagram"></center>

The motor applies a torque that twists the arm against a linear spring and moves the end of the arm across a rotating platter. The input to the system is the motor torque $\tau_\text{m}$. The force exerted by the spring is a nonlinear function of the head position due to the way it is attached.

The equations of motion for the system are given by

$$
J \ddot \theta = -b \dot\theta - k r\sin\theta + \tau_\text{m},
$$

which can be written in state space form as

$$
\frac{d}{dt} \begin{bmatrix} \theta \\ \theta \end{bmatrix} =
  \begin{bmatrix} \dot\theta \\ -k r \sin\theta / J - b\dot\theta / J \end{bmatrix}
  + \begin{bmatrix} 0 \\ 1/J \end{bmatrix} \tau_\text{m}.
$$

The system parameters are given by

$$
k = 1,\quad J = 100,\quad b = 10,
\quad r = 1,\quad l = 2,\quad \epsilon = 0.01.
$$

and we assume that time is measured in milliseconds (ms) and distance in centimeters (cm).  (The constants here are made up and don't necessarily reflect a real disk drive, though the units and time constants are motivated by computer disk drives.)

In [ ]:
# Parameter values
servomech_params = {
    'J': 100,             # Moment of inertia of the motor
    'b': 10,              # Angular damping of the arm
    'k': 1,               # Spring constant
    'r': 1,               # Location of spring contact on arm
    'l': 2,               # Distance to the read head
    'eps': 0.01,          # Magnitude of velocity-dependent perturbation
}

# State derivative
def servomech_update(t, x, u, params):
    # Extract the configuration and velocity variables from the state vector
    theta = x[0]                # Angular position of the disk drive arm
    thetadot = x[1]             # Angular velocity of the disk drive arm
    tau = u[0]                  # Torque applied at the base of the arm

    # Get the parameter values
    J, b, k, r = map(params.get, ['J', 'b', 'k', 'r'])

    # Compute the angular acceleration
    dthetadot = 1/J * (
        -b * thetadot - k * r * np.sin(theta) + tau)

    # Return the state update law
    return np.array([thetadot, dthetadot])

# System output (full state)
def servomech_output(t, x, u, params):
    l = params['l']
    return l * x[0]

# System dynamics
servomech = ct.nlsys(
    servomech_update, servomech_output, name='servomech',
    params=servomech_params,
    states=['theta_', 'thdot_'],
    outputs=['y'], inputs=['tau'])

In addition to the system dynamics, we assume there are actuator dynamics that limit the performance of the system.  We take these as first order dynamics with saturation:

$$
\tau = \text{sat} \left(\frac{\alpha}{s + \alpha} u\right)
$$

In [ ]:
actuator_params = {
    'umax': 5,            # Saturation limits
    'alpha': 10,          # Actuator time constant
}

def actuator_update(t, x, u, params):
  # Get parameter values
  alpha = params['alpha']
  umax = params['umax']

  # Clip the input
  u_clip = np.clip(u, -umax, umax)

  # Actuator dynamics
  return -alpha * x + alpha * u_clip

actuator = ct.nlsys(
    actuator_update, None, params=actuator_params,
    inputs='u', outputs='tau', states=1, name='actuator')

system = ct.series(actuator, servomech)
system.name = 'system'  # missing feature
print(system)

### Linearization

To study the open loop dynamics of the system, we compute the linearization of the dynamics about the equilibrium point corresponding to $\theta_\text{e} = 15^\circ$.

In [ ]:
# Convert the equilibrium angle to radians
theta_e = (15 / 180) * np.pi

# Compute the input required to hold this position
u_e = servomech.params['k'] * servomech.params['r'] * np.sin(theta_e)
print("Equilibrium torque = %g" % u_e)

# Linearize the system dynamics about the equilibrium point
P = ct.tf(
    system.linearize([0, theta_e, 0], u_e, copy_names=True)[0, 0])
P.name = 'P'  # bug
print(P, end="\n\n")

ct.bode_plot(P)

## Ziegler-Nichols tuning

Ziegler-Nichols tuning provides a method for choosing the gains of a PID controller that give reasonable closed loop response.  More information can be found in [Feedback Systems](https://fbswiki.org/wiki/index.php/Feedback_Systems:_An_Introduction_for_Scientists_and_Engineers) (FBS2e), Section 11.3.

We show here the figures and tables that we will use (from FBS2e):

<center>
<table>
<tr>
<td align='middle'>
<img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/zn-step-response.png" width=300>
</td>
<td align='middle'>
<img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/zn-step-table.png" width=300>
</td>
</tr>
</center>

To use the Ziegler-Nichols turning rules, we plot the step response, compute the parameters (shown in the figure), and then apply the formulas in the table:

In [ ]:
# Plot the step response
resp = ct.step_response(P)
resp.plot()

# Find the point of the steepest slope
slope = np.diff(resp.outputs) / np.diff(resp.time)
mxi = np.argmax(slope)
mx_time = resp.time[mxi]
mx_out= resp.outputs[mxi]
plt.plot(resp.time[mxi], resp.outputs[mxi], 'ro')

# Draw a line going through the point of max slope
mx_slope = slope[mxi]
timepts = np.linspace(0, mx_time*2)
plt.plot(timepts, mx_out + mx_slope * (timepts - mx_time), 'r-')

# Solve for the Ziegler-Nichols parameters
a = -(mx_out - mx_slope * mx_time)  # Find the value of the line at t = 0
tau = a / mx_slope                  # Solve a + mx_slope * tau = 0
print(f"{a=}, {tau=}")

We can then construct a controller using the parameters:

In [ ]:
s = ct.tf('s')

# Proportional controller
kp = 1/a
ctrl_zn_P = kp

# PI controller
kp = 0.9/a
Ti = tau/0.3; ki = kp/Ti
ctrl_zn_PI = kp + ki / s

# PID controller
kp = 1.2/a
Ti = tau/0.5; ki = kp/Ti
Td = 0.5 * tau; kd = kp * Td
ctrl_zn_PID = kp + ki / s + kd * s

print(ctrl_zn_PID)

In [ ]:
# Compute the closed loop systems and plots the step and
# frequency responses.

clsys_zn_P = ct.feedback(P * ctrl_zn_P)
clsys_zn_P.name = 'P'

clsys_zn_PI = ct.feedback(P * ctrl_zn_PI)
clsys_zn_PI.name = 'PI'

clsys_zn_PID = ct.feedback(P * ctrl_zn_PID)
clsys_zn_PID.name = 'PID'

# Plot the step responses
resp.sysname = 'open_loop'
resp.plot(color='k')

stepresp_zn_P = ct.step_response(clsys_zn_P)
stepresp_zn_P.plot(color='b')

stepresp_zn_PI = ct.step_response(clsys_zn_PI)
stepresp_zn_PI.plot(color='r')

stepresp_zn_PID = ct.step_response(clsys_zn_PID)
stepresp_zn_PID.plot(color='g')
plt.legend()

plt.figure()
ct.bode_plot([clsys_zn_P, clsys_zn_PI, clsys_zn_PID]);

## Loop shaping

A better design can be obtained by looking at the loop transfer function and adjusting the controller parameters to give a loop shape that will give closed loop properties.  We show the steps for such a design here:

In [ ]:
# Design parameters
Td = 1                    # Set to gain crossover frequency
Ti = Td * 10              # Set to low frequency region
kp = 500                  # Tune to get desired bandwith

# Updated gains
kp = 150
Ti = Td * 5; kp = 150

# Compute controller parmeters
ki = kp/Ti
kd = kp * Td

# Controller transfer function
ctrl_shape = kp + ki / s + kd * s
ctrl_shape.name = 'C'

# Frequency response (open loop) - use this to help tune your design
ltf_shape = P * ctrl_shape
ltf_shape.name = 'L'

ct.frequency_response([P, ctrl_shape]).plot()
ct.frequency_response(ltf_shape).plot(margins=True);

In [ ]:
# Compute the closed loop systemsand plot the step response
# and Nyquist plot (to make sure margins look OK)

# Create the closed loop systems
clsys_shape = ct.feedback(P * ctrl_shape)
clsys_shape.name = 'loopshape'

# Step response
plt.subplot(2, 1, 1)
stepresp_shape = ct.step_response(clsys_shape)
stepresp_shape.plot(color='b')
plt.plot([0, stepresp_shape.time[-1]], [1, 1], 'k--')

# Compare to the ZN controller
ax = plt.subplot(2, 1, 2)
ct.step_response(clsys_shape, stepresp_zn_PID.time).plot(
    color='b', ax=np.array([[ax]]))
stepresp_zn_PID.plot(color='g', ax=np.array([[ax]]))
ax.plot([0, stepresp_shape.time[-1]], [1, 1], 'k--')

# Nyquist plot
plt.figure()
ct.nyquist([ltf_shape])

We see that the loop shaping controller has better step response (faster rise and settling time, less overshoot).

### Gang of Four

When designing a controller, it is important to look at all of the input/output responses, not just the response from reference to output (which is what the step response above focuses on). 

In the frequency domain, the Gang of 4 plots provide useful information on all (important) input/output pairs:

In [ ]:
ct.gangof4(P, ctrl_shape)

These all look pretty resonable, except that the transfer function from the reference $r$ to the system input $u$ is getting large at high frequency.  This occurs because we did not filter the derivative on the PID controller, so high frequency components of the reference signal (or the measurement noise!) get amplified.  We will fix this in the more advanced controller below.

## Anti-windup + derivative filtering

In addition to the amplification of high frequency signals due to the derivative term, another practical consideration in the use of PID controllers is integrator windup.  Integrator windup occurs when there are limits on the control inputs so that the error signal may not descrease quickly.  This causes the integral term in the PID controller to see an error for a long period of time, and the resulting integration of the error must be offset by making the error have opposite sign for some period of time.  This is often undesireable.

To see how to address both amplification of noise due to the derivative term and integrator windup effects in the presence of input constraints, we now implement PID controller with anti-windup and derivative filtering, as shown in the following figure (see also Figure 11.11 in [FBS2e](https://fbswiki.org/wiki/index.php/Feedback_Systems:_An_Introduction_for_Scientists_and_Engineers)):

<center>
<img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/pid-aw-diagram.png"</img>
</center>

### Low pass filter

The low pass filtered derivative has transfer function

$$
G(s) = \frac{a\, s}{s + a}.
$$

This can be implemented using the differential equation

$$
\dot \xi = -a \xi + a y, \qquad
\eta = -a \xi + a y.
$$

In [ ]:
ctrl_params = {'kaw': 5 * ki, 'a': 10/Td}

def ctrl_update(t, x, u, params):
  # Get the parameter values
  kaw = params['kaw']
  a = params['a']
  umax_ctrl = params.get('umax_ctrl', actuator.params['umax'])

  # Extract the signals into more familiar variable names
  r, y = u[0], u[1]
  z = x[0]        # integral error
  xi = x[1]       # filtered derivative

  # Compute the controller components
  u_prop = kp * (r - y)
  u_int = z
  ydt_f = -a * xi + a * (-y)
  u_der = kd * ydt_f

  # Compute the commanded and saturated outputs
  u_cmd = u_prop + u_int + u_der
  u_sat = np.clip(u_cmd, -umax_ctrl, umax_ctrl)

  dz = ki * (r - y) + kaw * (u_sat - u_cmd)
  dxi = -a * xi + a * (-y)
  return np.array([dz, dxi])

def ctrl_output(t, x, u, params):
  # Get the parameter values
  kaw = params['kaw']
  a = params['a']
  umax_ctrl = params.get('umax_ctrl', params['umax'])

  # Extract the signals into more familiar variable names
  r, y = u[0], u[1]
  z = x[0]        # integral error
  xi = x[1]       # filtered derivative

  # Compute the controller components
  u_prop = kp * (r - y)
  u_int = z
  ydt_f = -a * xi + a * (-y)
  u_der = kd * ydt_f

  # Compute the commanded and saturated outputs
  u_cmd = u_prop + u_int + u_der
  u_sat = np.clip(u_cmd, -umax_ctrl, umax_ctrl)

  return u_cmd

ctrl = ct.nlsys(
    ctrl_update, ctrl_output, name='ctrl', params=ctrl_params,
    inputs=['r', 'y'], outputs=['u'], states=2)

clsys = ct.interconnect(
    [servomech, actuator, ctrl], name='clsys',
    inputs=['r'], outputs=['y', 'tau'])
print(clsys)

In [ ]:
# Plot the step responses for the following cases:
#
# 'linear': the original linear step response (no actuation limits)
# 'clipped': PID controller with input limits, but not anti-windup
# 'anti-windup': PID controller with anti-windup compensation

# Use more time points to get smoother response curves
timepts = np.linspace(0, 2*stepresp_shape.time[-1], 500)

# Compute the response for the individual cases
stepsize = theta_e / 2
resp_ln = ct.input_output_response(
    clsys, timepts, stepsize, params={'umax': np.inf, 'kaw': 0, 'a': 1e3})
resp_cl = ct.input_output_response(
    clsys, timepts, stepsize, params={'umax': 5, 'kaw': 0, 'a': 100})
resp_aw = ct.input_output_response(
    clsys, timepts, stepsize, params={'umax': 5, 'kaw': 2*ki, 'a': 100})

# Plot the time responses in a single plot
ct.time_response_plot(resp_ln, color='b', plot_inputs=False, label="linear")
ct.time_response_plot(resp_cl, color='r', plot_inputs=False, label="clipped")
ct.time_response_plot(resp_aw, color='g', plot_inputs=False, label="anti-windup");

The response of the anti-windup compensator is very sluggish, indicating that we may be setting $k_\text{aw}$ too high.

In [ ]:
resp_aw = ct.input_output_response(
    clsys, timepts, stepsize, params={'umax': 5, 'kaw': 0.05 * ki, 'a': 100})

# Plot the time responses in a single plot
ct.time_response_plot(resp_ln, color='b', plot_inputs=False, label="linear")
ct.time_response_plot(resp_cl, color='r', plot_inputs=False, label="clipped")
ct.time_response_plot(resp_aw, color='g', plot_inputs=False, label="anti-windup");

This gives a much better response, though the value of $k_\text{aw}$ falls well outside the range of [2, 10].  One reason for this is that $k_\text{aw}$ acts on the inputs, $\tau$, which are roughly 100 larger than the size of the outputs, $y$, as seen in the above plots.

We can now see if this affects the Gang of Four in the expected way:

In [ ]:
C = ctrl.linearize([0, 0], 0, params=resp_aw.params)[0, 1]
ct.gangof4(P, C);

Note that in the transfer function from $r$ to $u$ (which is the same as the transfer function from $e$ to $u$, the high frequency gain is now bounded.  (We could make it go back down by using a second order filter.)